In [ ]:
import sys
import datetime as dt
from getpass import getuser

import pandas as pd
from sqlalchemy import text

sys.path.append('C:\Users\user\Desktop\github\sotrans_analytics_department')
from source.core.de.serving.datamart.modules.config import DEFAULT_ENGINE

In [ ]:
ROOT_FOLDER: str = r'C:\Users\user\Desktop\github\sotrans_analytics_department'
TRADE_FOLDER: str = r'\\192.168.101.228\Trade'

In [ ]:
CURRENT_ORDERS_FILE: str = rf'{TRADE_FOLDER}\2025\Orders\Подтвержденные заказы 2025.xlsx'

In [ ]:
# Сформировать датафрейм по текущим размещённым заказам;
orders: pd.DataFrame = pd.read_excel(
    io=CURRENT_ORDERS_FILE,
    engine='openpyxl',
    usecols=[1, 7, 17, 20, 26, 27],
    skiprows=1
)

In [ ]:
# Переименование столбцов;
orders: pd.DataFrame = (
    orders
    .rename(
        columns={
            'Код 1С': 'sku_id_1c',
            'Заказано кол-во ': 'ordered_cnt',
            'Бэкордер кол-во': 'backorder_cnt',
            'Отгруж кол-во': 'shipped_cnt',
            'Текущий статус': 'order_status',
            'Ожидаемая дата прихода на склад': 'arrival_date'
        }
    )
)

In [ ]:
orders.info()

In [ ]:
# Фильтрация пропусков;
orders: pd.DataFrame = orders.dropna(subset='sku_id_1c')
orders: pd.DataFrame = orders[orders['sku_id_1c'] != '        ']

In [ ]:
# Приведение к нижнему регистру;
orders['sku_id_1c'] = orders['sku_id_1c'].str.lower()
orders['order_status'] = orders['order_status'].str.lower()

In [ ]:
# Замена пропусков в shipped_cnt;
orders.loc[:, 'shipped_cnt'] = orders['shipped_cnt'].fillna(value=0)

In [ ]:
# Замена пропусков в order_status;
orders.loc[:, 'order_status'] = orders['order_status'].fillna(value='_нет данных')

In [ ]:
# Замена пропусков в shipped_cnt;
new_arrival_date = pd.Timestamp.today() + pd.DateOffset(days=180)
orders.loc[:, 'arrival_date'] = orders['arrival_date'].fillna(value=new_arrival_date.date())

In [ ]:
# Группировака и аггрегация
orders: pd.DataFrame = (
    orders
    .groupby(
        by=[
            'sku_id_1c',
            'order_status',
            'arrival_date'
        ]
    )
    .aggregate(
        {
            'ordered_cnt': 'sum',
            'backorder_cnt': 'sum',
            'shipped_cnt': 'sum'
        }
    )
    .reset_index()
)

In [ ]:
orders.loc[:, 'last_update_at'] = pd.Timestamp.today()

In [ ]:
orders.info()

# Дополнительная проверка

In [ ]:
break

In [ ]:
print(*orders['order_status'].unique(), sep='\n')

In [ ]:
orders[orders['arrival_date'] < dt.datetime.today()]

# SAVE DATA

In [ ]:
with DEFAULT_ENGINE.begin() as connection:
    connection.execute(text("TRUNCATE TABLE report.cr_orders;"))
    
    orders.to_sql(
        name='cr_orders',
        con=connection,
        schema='report',
        index=False,
        if_exists='append'
    )